In [ ]:
from pathlib import Path
import re
import pandas as pd
import holoviews as hv
from holoviews import opts

hv.extension("bokeh")

In [ ]:
opts.defaults(opts.BoxWhisker(height=1200, width=1200))

In [ ]:
log = Path("../dbt_logs/dbt.log").read_text()

In [ ]:
pattern = re.compile(
    r"created sql incremental|table model prod.([a-z_]+).+in ([0-9\.]+)s",
    flags=re.IGNORECASE,
)

r = pattern.findall(log)

df = pd.DataFrame(r, columns=["model", "build_time"])

df.build_time = df.build_time.replace("", None).astype(float)

df = df.sort_values("build_time", ascending=False)

df["build_time_minutes"] = df.build_time / 60

In [ ]:
df.groupby("model").build_time.describe()

In [ ]:
p = hv.BoxWhisker(
    df[df.build_time >= 60], kdims=["model"], vdims=["build_time_minutes"]
)
p.opts(
    opts.BoxWhisker(
        invert_axes=True,
        show_grid=True,
        title="Build times for models in dbt tranformation pipeline",
    )
)

In [ ]:
log_source = Path("../dbt_logs/dbt_source.log").read_text()
pattern = re.compile(
    r"created sql incremental|table model source.([a-z_]+).+in ([0-9\.]+)s",
    flags=re.IGNORECASE,
)
r = pattern.findall(log_source)

df = pd.DataFrame(r, columns=["model", "build_time"])

df.build_time = df.build_time.replace("", None).astype(float)

df = df.sort_values("build_time", ascending=False)

df["build_time_minutes"] = df.build_time / 60

In [ ]:
len(r)

In [ ]:
df.groupby("model").build_time.describe()

In [ ]:
p = hv.BoxWhisker(
    df[df.build_time >= 30], kdims=["model"], vdims=["build_time_minutes"]
)
p.opts(
    opts.BoxWhisker(
        invert_axes=True,
        show_grid=True,
        title="Build times for models in dbt extract-load pipeline (>=0.5minutes)",
    )
)

## Timing info

In [ ]:
pattern = re.compile(
    r"Timing info for model.dbt_omop.([a-z_]+) \(execute\): (.+) => (.+)",
    flags=re.IGNORECASE,
)

In [ ]:
r = pattern.findall(log)

df = pd.DataFrame(r, columns=["model", "start_time", "end_time"])

df.start_time = pd.to_datetime(df.start_time)
df.end_time = pd.to_datetime(df.end_time)

df["start_date"] = df.start_time.dt.date

dbt_start_time = (
    df.groupby("start_date").start_time.min().rename("dbt_start_time").reset_index()
)

df = df.merge(dbt_start_time, on="start_date")

df["start_time_offset"] = (df.start_time - df.dbt_start_time).dt.seconds / 60
df["end_time_offset"] = (df.end_time - df.dbt_start_time).dt.seconds / 60

df["build_time"] = (df.end_time - df.start_time).dt.seconds

df

In [ ]:
x = df.groupby("model")[["start_time_offset", "end_time_offset", "build_time"]].mean()

x = x.reset_index()
x["model_sort_order"] = x.model.apply(lambda x: x.split("__")[-1])


x = x.sort_values(["end_time_offset", "model_sort_order", "model"])

In [ ]:
p = hv.Segments(
    x.loc[(~x.model.str.contains("src_")) & (x.end_time_offset < 180)],
    kdims=["start_time_offset", "model", "end_time_offset", "model"],
)
p.opts(opts.Segments(width=1200, height=1800, line_width=10, show_grid=True))